*   Author : Ziang Xu
*   Student number : 180104048
*   Code : Resnet-50 model construction, training and testing.
*   Grad-CAM Visualization.

In [0]:
# Google Colaboratory carry Google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Enter the classification task directory
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/ML/classification/")
!ls

In [0]:
!pip uninstall tensorflow
!pip uninstall keras
!pip uninstall keras-vis


In [0]:
!pip install keras
!pip install tensorflow==1.12
!pip install git+https://github.com/raghakot/keras-vis.git

In [0]:
# Read dataset
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
images=[]
labels=[]
def read_image(imageName):
    im = Image.open(imageName).convert('RGB')
    data = np.array(im)
    return data

In [0]:
# Read there are several folders in the dataset.
text = os.listdir('./resize_data')
# Save the name of the image in the folder and its corresponding folder to the corresponding list.
for textPath in text:
    for fn in os.listdir(os.path.join('resize_data', textPath)):
        if fn.endswith('.jpg'):
            fd = os.path.join('./resize_data', textPath, fn)
            images.append(read_image(fd))
            labels.append(textPath)
X = np.array(images)
y = np.array(list(map(int, labels)))             


In [0]:
# Split dataset to 0.8 for training and 0.2 for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)
# Normalized dataset
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 8)
Y_test = np_utils.to_categorical(y_test, 8)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [0]:
# Fine-tuning technique is used to build model.
from keras.applications.resnet50 import ResNet50
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import SGD

base_model = ResNet50(weights = 'imagenet',include_top = False,pooling = 'avg',
                      input_shape=(224, 224, 3))


predictions = Dense(4096, activation='relu')(base_model.output)
predictions = Dense(4096, activation='relu')(base_model.output)
predictions = Dropout(0.5)(predictions)
predictions = Dense(8, activation='softmax')(predictions)



model = Model(inputs=base_model.input, outputs=predictions, name='Resnet50')
sgd = SGD(lr=0.01, decay=1e-5)

In [0]:
# Compile the model.
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
print('Training begin')
history=model.fit(X_train, y_train, batch_size=50, epochs=100,validation_data=(X_test, Y_test))

In [0]:
# Draw a graph of accuracy and loss functions.
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# Model saving and loading.
# model.save('model2.h5')
from keras.models import load_model
model=load_model('model2.h5')

In [0]:
# Model visualization
from keras.applications import ResNet50
from vis.utils import utils
from keras import activations

# Hide warnings on Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')

layer_idx = utils.find_layer_idx(model, 'dense_5')

model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

In [0]:
from vis.utils import utils
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

img1 = utils.load_img('image/1.jpg', target_size=(224, 224))
f, ax = plt.subplots(1, 1)
ax.imshow(img1)

In [0]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations

# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'dense_5')

f, ax = plt.subplots(1, 1)  

grads = visualize_saliency(model, layer_idx, filter_indices=0, seed_input=img1)
    
# visualize grads as heatmap
ax.imshow(grads, cmap='jet')

In [0]:
for modifier in ['guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 1)
    plt.suptitle(modifier)
       
    
    grads = visualize_saliency(model, layer_idx, filter_indices=0, 
                                   seed_input=img1, backprop_modifier=modifier)
    # Lets overlay the heatmap onto original image.    
    ax.imshow(grads, cmap='jet')

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

penultimate_layer = utils.find_layer_idx(model, 'res5c_branch2c')

for modifier in [None, 'guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 1)
    plt.suptitle("vanilla" if modifier is None else modifier)
      
    # 0 is the imagenet index corresponding to `dyed-lifted-polyps`
    grads = visualize_cam(model, layer_idx, filter_indices=0, 
                              seed_input=img1, penultimate_layer_idx=penultimate_layer,
                              backprop_modifier=modifier)        
    # Lets overlay the heatmap onto original image.    
    jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
    ax.imshow(overlay(jet_heatmap, img1))